# DC Resistivity: 1D parametric inversion 
_Inverting for Resistivities and Layers_


Here we use the module *SimPEG.electromangetics.static.resistivity* to invert
DC resistivity sounding data and recover the resistivities and layer thicknesses
for a 1D layered Earth. In this tutorial, we focus on the following:

- How to define sources and receivers from a survey file
- How to define the survey
- Defining a model that consists of resistivities and layer thicknesses

For this tutorial, we will invert sounding data collected over a layered Earth using
a Wenner array. The end product is layered Earth model which explains the data.
    

## Import modules

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from discretize import TensorMesh

from SimPEG import (
    maps, data, data_misfit, regularization,
    optimization, inverse_problem, inversion, directives
)
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG.electromagnetics.static.utils.StaticUtils import plot_layer

mpl.rcParams.update({'font.size': 14})

## Define Parameters for the Inversion

In [ ]:
# Define the file path to the data file. Also define the AB/2, MN/2 and apparent resistivity columns.
# Recall that python counts starting at 0
data_filename = './sounding_data/Aung_San_Location_1_raw.csv'
half_AB_column = 'AB/2 (m)'
half_MN_column = 'MN/2 (m)'
apparent_resistivity_column = 'App. Res. (Ohm m)'

# Define the floor and percent uncertainty you would like to apply to apparent resistivity data
uncertainty_floor = 5
uncertainty_percent = 10.

# Define layer thicknesses and resistivities for the starting model. The thickness
# of the bottom layer is not used, as we assume it extends downward to infinity.
layer_thicknesses = np.r_[10, 10]
halfspace_resistivity = 300.

## Load Data, Define Survey and Plot

Here we load the observed data, define the DC survey geometry and plot the
data values.

In [ ]:
# Load data
df = pd.read_csv(data_filename)

# Extract source and receiver electrode locations and the observed data
half_AB_separations = df[half_AB_column]
half_MN_separations = df[half_MN_column]
dobs = df[apparent_resistivity_column].values

resistivities = halfspace_resistivity*np.ones(layer_thicknesses.size+1)

# Define survey
unique_tx, k = np.unique(half_AB_separations, return_index=True)
n_sources = len(k)
k = np.sort(k)
k = np.r_[k, len(dobs)+1]

source_list = []
for ii in range(0, n_sources):
    
    # MN electrode locations for receivers. Each is an (N, 3) numpy array
    M_locations = -half_MN_separations[k[ii]:k[ii+1]]
    M_locations = np.c_[M_locations, np.zeros((np.shape(M_locations)[0], 2))]
    N_locations = half_MN_separations[k[ii]:k[ii+1]]
    N_locations = np.c_[N_locations, np.zeros((np.shape(N_locations)[0], 2))]
    receiver_list = [dc.receivers.Dipole(M_locations, N_locations)]
    
    # AB electrode locations for source. Each is a (1, 3) numpy array
    A_location = np.r_[-half_AB_separations[k[ii]], 0., 0.]
    B_location = np.r_[half_AB_separations[k[ii]], 0., 0.]
    source_list.append(dc.sources.Dipole(receiver_list, A_location, B_location))

# Define survey
survey = dc.Survey(source_list)

# Compute the A, B, M and N electrode locations.
survey.getABMN_locations()

# Plot apparent resistivities on sounding curve as a function of Wenner separation
# parameter.
electrode_separations = np.sqrt(
    np.sum((survey.m_locations - survey.n_locations)**2, axis=1)
)

fig, ax = plt.subplots(1, 1, figsize=(11, 5))
ax.loglog(half_AB_separations, dobs, 'b', lw=2)
ax.grid(True, which='both', ls="--", c='gray')
ax.set_xlabel("AB/2 (m)")
ax.set_ylabel("Apparent Resistivity ($\Omega m$)")

## Assign Uncertainties

Inversion with SimPEG requires that we define uncertainties on our data. The
uncertainty represents our estimate of the standard deviation of the noise on
our data.

In [ ]:
uncertainties = uncertainty_floor + 0.01*uncertainty_percent*np.abs(dobs)

## Define Data


Here is where we define the data that are inverted. The data are defined by
the survey, the observation values and the uncertainties.

In [ ]:
data_object = data.Data(survey, dobs=dobs, standard_deviation=uncertainties)

## Defining the Starting Model and Mapping


In [ ]:
# Define the layers as a mesh
mesh = TensorMesh([layer_thicknesses], '0')
print(mesh)

# Define model. We are inverting for the layer resistivities and layer thicknesses.
# Since the bottom layer extends to infinity, it is not a model parameter for
# which we need to invert. For a 3 layer model, there is a total of 5 parameters.
# For stability, our model is the log-resistivity and log-thickness.
starting_model = np.r_[np.log(resistivities), np.log(layer_thicknesses)]

# Since the model contains two different properties for each layer, we use
# wire maps to distinguish the properties.
wire_map = maps.Wires(('rho', mesh.nC+1), ('t', mesh.nC))
resistivity_map = maps.ExpMap(nP=mesh.nC+1) * wire_map.rho
layer_map = maps.ExpMap(nP=mesh.nC) * wire_map.t

## Define the Physics

Here we define the physics of the problem using the DCSimulation_1D class.


In [ ]:
simulation = dc.simulation_1d.Simulation1DLayers(
    survey=survey, rhoMap=resistivity_map, thicknessesMap=layer_map,
    data_type="apparent_resistivity"
)

## Define Inverse Problem

The inverse problem is defined by 3 things:

    1) Data Misfit: a measure of how well our recovered model explains the field data
    2) Regularization: constraints placed on the recovered model and a priori information
    3) Optimization: the numerical approach used to solve the inverse problem



In [ ]:
# Define the data misfit. Here the data misfit is the L2 norm of the weighted
# residual between the observed data and the data predicted for a given model.
# The weighting is defined by the reciprocal of the uncertainties.
dmis = data_misfit.L2DataMisfit(simulation=simulation, data=data_object)

# Define the regularization on the parameters related to resistivity
mesh_rho = TensorMesh([mesh.hx.size+1])
reg_rho = regularization.Simple(
    mesh_rho, alpha_s=1., alpha_x=1,
    mapping=wire_map.rho
)

# Define the regularization on the parameters related to layer thickness
mesh_t = TensorMesh([mesh.hx.size])
reg_t = regularization.Simple(
    mesh_t, alpha_s=1., alpha_x=1,
    mapping=wire_map.t    
)

# Combine to make regularization for the inversion problem
reg = reg_rho + reg_t

# Define how the optimization problem is solved. Here we will use an inexact
# Gauss-Newton approach that employs the conjugate gradient solver.
opt = optimization.InexactGaussNewton(
    maxIter=20, maxIterCG=30, print_type='ubc'
)

# Define the inverse problem
inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)

## Define Inversion Directives

Here we define any directives that are carried out during the inversion. This
includes the cooling schedule for the trade-off parameter (beta), stopping
criteria for the inversion and saving inversion results at each iteration.



In [ ]:
# Defining a starting value for the trade-off parameter (beta) between the data
# misfit and the regularization.
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1.)

# Set the rate of reduction in trade-off parameter (beta) each time the
# the inverse problem is solved. And set the number of Gauss-Newton iterations
# for each trade-off paramter value.
beta_schedule = directives.BetaSchedule(coolingFactor=2., coolingRate=1.)

# Apply and update sensitivity weighting as the model updates
update_sensitivity_weights = directives.UpdateSensitivityWeights()

# Options for outputting recovered models and predicted data for each beta.
save_iteration = directives.SaveOutputEveryIteration(save_txt=False)

# Setting a stopping criteria for the inversion.
target_misfit = directives.TargetMisfit(chifact=1)

# The directives are defined in a list
directives_list = [
    starting_beta, beta_schedule, target_misfit
]

## Running the Inversion


To define the inversion object, we need to define the inversion problem and
the set of directives. We can then run the inversion.




In [ ]:
# Here we combine the inverse problem and the set of directives
inv = inversion.BaseInversion(inv_prob, directives_list)

# Run the inversion
recovered_model = inv.run(starting_model)

In [ ]:
# Inversion result from Mon DRD Mawlamyine location 3
res_tmp = np.array([348.4, 722.9, 282, 100.8, 51.4, 170.8, 31.1, 184.3])
thick_tmp = np.array([1.4, 1.6, 1.4, 12.1, 11.4, 25.1, 54.2])
plotting_mesh_tmp = TensorMesh([np.r_[thick_tmp, layer_thicknesses[-1]]], '0')

## Examining the Results



In [ ]:
# Plot true model and recovered model
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
plotting_mesh = TensorMesh([np.r_[layer_map*recovered_model, layer_thicknesses[-1]]], '0')
x_min = np.min(resistivity_map*recovered_model)
x_max = np.max(resistivity_map*recovered_model)

plot_layer(resistivity_map*recovered_model, plotting_mesh, ax=ax, depth_axis=False, color='k')
#plot_layer(res_tmp, plotting_mesh_tmp, ax=ax, depth_axis=False, color='r')
#ax.set_xlim(10, 5000)
#ax.set_ylim(-300, 0)
#ax.legend(("SimPEG", "Mon State DRD"))
ax.grid(True, which='both', ls="--", c='gray')

In [ ]:
# Plot the true and apparent resistivities on a sounding curve
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.loglog(half_AB_separations, dobs, 'kx', lw=2, ms=10, mew=2)
ax.loglog(half_AB_separations, inv_prob.dpred, 'k', lw=2)
ax.set_xlabel("AB/2 (m)")
ax.set_ylabel("Apparent Resistivity ($\Omega m$)")
ax.legend(['Observed data','Predicted data'])
#ax.set_ylim(50, 1000)
ax.grid(True, which='both')